In [1]:
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports
import base64
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter class from your CRUD Python module
from animal_shelter import NewAnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
client = "localhost" # Using a local host with MongoDB Compass
port = 27017 # New port is 27017
database = "AAC"
collection = "animals"

# Initialize the AnimalShelter object
db = NewAnimalShelter(username, password, client, port, database, collection)

# Read data from the database
df = pd.DataFrame.from_records(db.read({}))

# Drop the '_id' column to avoid issues with Dash DataTable
df.drop(columns=['_id'], inplace=True)

# Debug: Print the dataframe
print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

# Create unique identifier which contains my name
unique_identifier = "Sierra Baltins"

# Load Grazioso Salvare Logo.png
image_filename = r'C:\Users\heart\Downloads\Grazioso Salvare Logo.jpeg'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')

app.layout = html.Div([
    # Implement Logo
    html.A(
        html.Img(src=f'data:image/jpeg;base64,{encoded_image}',
                 alt='Grazioso Salvare Logo',
                 # Adjust style as needed
                 style={'height': '100px'}),  
        # Add website or anchor tag to Grazioso Salvare Logo
        href='http://www.snhu.edu',
        target='_blank'
    ),
    
    # Implement unique identifier with name
    html.Div(f"Dashboard created by: {unique_identifier}"),
    
    # Input field for the username
    dcc.Input(
        id="input_user",
        type="text",
        placeholder="username"
    ),
    
    # Input field for the password
    dcc.Input(
        id="input_passwd",
        type="password",  # Using password type for security
        placeholder="password"
    ),
    
    # Create a button labeled 'Submit'
    html.Button('Submit', id='submit-val', n_clicks=0),
    
    # Generate a horizontal line separating input from output element
    html.Hr(),
    
    # Output element for the dashboard
    html.Div(id="query-out", style={'whiteSpace': 'pre-line'}),
    
    html.Div(id="query-out-filter"),
    
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Add title of the dashboard and center it
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),

    html.Div([
        # Add string to prompt user to select rescue type option
        html.Center(html.Label(['Filter animals by rescue type:'], style={'font-weight': 'bold'})),
        # Add radio options for Water Rescue, Mountain Rescue, Disaster Rescue, or Reset
        html.Center(dcc.RadioItems(
            id='radio-items',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Filter table by rescue options',
            inputStyle={"margin-left": "20px"},
            labelStyle={'display': 'inline-block'}
        ))
    ]),
    html.Br(),

    # Create a data table and implement its features
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
       # set data table interactivity
        editable=False,
        filter_action="native",
        sort_action='custom',
        sort_mode="multi",
        sort_by=[],
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    # Adjust the placement of the data table, pie graph, and the geo location map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Update the data table and filter animals when the user selects a radio option
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('radio-items', 'value')]
)
def update_dashboard(radio_items):
    # Implement a query to find specific data from AAC.csv
    query = {}

    # Return certain animals on the data table if the Water Rescue radio option is clicked on
    if radio_items == 'Water Rescue':
        query = {
            "Animal Type": "Dog",
            "Breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "Sex upon Outcome": "Intact Female",
            #"age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }

    # Return certain animals on the data table if the Mountain Rescue radio option is clicked on
    elif radio_items == 'Mountain Rescue':
        query = {
            "Animal Type": "Dog",
            "Breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "Sex upon Outcome": "Intact Male",
            #"age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }

    # Return certain animals on the data table if the Disaster Rescue radio option is clicked on
    elif radio_items == 'Disaster Rescue':
        query = {
            "Animal Type": "Dog",
            "Breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "Sex upon Outcome": "Intact Male",
            #"age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }

    # Perform the query and create the filtered DataFrame
    df_filtered = pd.DataFrame(list(db.read(query)))

    # Print the filtered DataFrame
    print(f"Query: {query}")
    print(df_filtered.head())

    # Check if the DataFrame is empty
    if df_filtered.empty:
        # Return the original data and columns if the filter returns no data
        return df.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    
    # Drop the '_id' column if it exists
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)

    return df_filtered.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns]


# Implement the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    ])

# The pie chart will update depending on how the data table presents its data (filtered results)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(    
            # The pie chart will show the ratio and percentage of different breeds in the data table
            figure = px.pie(dff, names='Breed', title='Distribution of Breeds')
        )    
    ]

# Implement the geolocation map    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)

def update_map(viewData, selected_rows):
    if not viewData:
        # Returns a default map centered on Austin, TX
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()])]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Default to the first row if no row is selected
    row = selected_rows[0] if selected_rows else 0

    # Ensure the selected row is within the DataFrame's bounds
    if row >= len(dff):
        row = 0

    # Define the latitude and longitude column names
    location_lat = 'location_lat'  
    location_long = 'location_long'  

    # Fetch the latitude and longitude from the DataFrame
    latitude = dff.iloc[row][location_lat]
    longitude = dff.iloc[row][location_long]

    # Fetch additional data for the popup
    animal_name = dff.iloc[row].get('name', 'Name')
    breed = dff.iloc[row].get('breed', 'Breed')

    # Return the updated geolocation map depending on which animal is selected in the data table
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P('Name')
                ])
            ])
        ])
    ]

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)

Index(['Animal ID', 'Name', 'DateTime', 'MonthYear', 'Date of Birth',
       'Outcome Type', 'Animal Type', 'Sex upon Outcome', 'Age upon Outcome',
       'Breed', 'Color', 'location_lat', 'location_long', 'Outcome Subtype'],
      dtype='object')
